In [ ]:
#%run NB01-Load.ipynb

In [ ]:
import collections
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py
import random 
#import seaborn as sb
import seaborn as sns
#import seaborn as snss
import sklearn
import statsmodels.api as sm
import warnings

from datetime import date
from IPython.display import HTML
from lightgbm import LGBMClassifier
from lightgbm import plot_importance
from pandas import DataFrame
from pandas import Series
from plotly import tools
from plotly.offline import init_notebook_mode
from plotly.offline import iplot
from pylab import rcParams
from sklearn import datasets
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
#from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from wordcloud import WordCloud

init_notebook_mode(connected=True)
plt.rcParams.update({'figure.max_open_warning': 200})
warnings.filterwarnings("ignore")

%matplotlib inline

#import C:/Development/kaggle--home-credit-default-risk/rand_jitter
#import C:/Development/kaggle--home-credit-default-risk/draw_feature_distribution
import sys
# sys.path.insert(0, 'C:/Development/kaggle--home-credit-default-risk/') # ~= sys.path.prepend
sys.path.append('C:/Development/kaggle--home-credit-default-risk/')
# import rand_jitter
# import draw_feature_distribution
##from rand_jitter import * # NOTE: added directly to draw_feature_distribution_v2
# from draw_feature_distribution import *
# from draw_feature_distribution_v1 import *
from draw_feature_distribution_v2 import *

# C:\Users\jbalcomb\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:
# The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.


In [4]:
path = "C:/Development/kaggle--home-credit-default-risk/data/"

In [5]:
application_train = pd.read_table(path + 'application_train.csv', sep=',', dtype=object)

In [ ]:
application_train.head()

In [ ]:
#application_train[other_columns] = ds13[other_columns].fillna(value = "MISSING")

In [6]:
application_train__ext_source = application_train.loc[:, ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','TARGET']]

In [7]:
# Remove observations with missing values
#application_train__ext_source.dropna(inplace = True)
application_train__ext_source__dropna = application_train__ext_source.dropna()

In [ ]:
#Create train and validation set
X_train, X_test, y_train, y_test = train_test_split(application_train__ext_source, application_train__ext_source['TARGET'], random_state=0)
#train_x, valid_x, train_y, valid_y = train_test_split(data, y, test_size=0.2, shuffle=True, stratify=y, random_state=1301)

In [ ]:
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

In [ ]:
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

In [ ]:
logreg = LogisticRegression()

In [ ]:
# Train model
model = logreg.fit(X_train, y_train)

In [ ]:
# Predict class
model.predict(X_test)

In [ ]:
# View predicted probabilities
model.predict_proba(X_test)

In [ ]:
logisticRegr = LogisticRegression()

In [ ]:
logisticRegr.fit(ds_train, convert_train.values.ravel())

In [ ]:
#print("Score=%.3f" % clf.score(X, grades["Letter"]))

In [ ]:
#print(logisticRegr.score(ds_test, convert_test))

In [ ]:
predictions = logisticRegr.predict(ds_test)

In [ ]:
print(logisticRegr.score(ds_test, convert_test))

In [ ]:
#cm = confusion_matrix(predictions, convert_test)

In [ ]:
#print(pd.DataFrame(cm, columns=labels, index=labels))
#print(pd.DataFrame(cm)

In [ ]:
seed = 1234567890

In [ ]:
kfold = model_selection.KFold(n_splits=10, random_state=seed)

In [ ]:
model = LogisticRegression(class_weight='balanced')

In [ ]:
scoring = 'accuracy'

In [ ]:
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)

In [ ]:
print("Accuracy: %.3f (%.3f)") % (results.mean(), results.std())

In [ ]:
print(results.mean())

In [ ]:
print(results.std())

In [ ]:
logit = sm.Logit(y_train, X_train)

In [ ]:
result = logit.fit(maxiter=999)

In [ ]:
print result.summary()

In [ ]:
#------------------------Build LightGBM Model-----------------------
train_data=lgb.Dataset(train_x,label=train_y)
valid_data=lgb.Dataset(valid_x,label=valid_y)